Cell 1: Install & Import

In [ ]:
!pip install plotly ipywidgets pandas --quiet

import requests
from datetime import datetime, timedelta
import plotly.graph_objects as go
from IPython.display import display, Image, HTML, FileLink
import csv
import pandas as pd
from ipywidgets import Dropdown, interact, widgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.2 MB/s eta 0:00:00


Cell 2: API & URLs

In [ ]:
API_KEY = "2284ff966e7f725235927f7acacc387a"
CURRENT_URL = "https://api.openweathermap.org/data/2.5/weather"
FORECAST_URL = "https://api.openweathermap.org/data/2.5/forecast"
ICON_URL = "http://openweathermap.org/img/wn/{}@2x.png"

cities = ["Dhaka", "Chittagong", "Khulna", "Sylhet", "Barishal", "London", "New York", "Tokyo", "Paris", "Sydney"]

Cell 3: Fetch Weather Function

In [ ]:
def get_weather(city):
    params = {"q": city, "appid": API_KEY, "units": "metric"}

    # Current weather
    res = requests.get(CURRENT_URL, params=params)
    if res.status_code != 200:
        print("❌ City not found or API error")
        return None

    data = res.json()
    current_temp = data["main"]["temp"]
    condition = data["weather"][0]["description"]
    humidity = data["main"]["humidity"]
    pressure = data["main"]["pressure"]
    wind_speed = data["wind"]["speed"]
    icon = data["weather"][0]["icon"]

    # Past 3 days (estimated)
    past_temps = [current_temp - (i*1.2) for i in range(3,0,-1)]

    # Next 7 days forecast
    future_temps = []
    future_conditions = []
    forecast_res = requests.get(FORECAST_URL, params=params)
    forecast_data = forecast_res.json()
    shown_dates = set()
    max_days = 7
    for item in forecast_data.get("list", []):
        date = item["dt_txt"].split(" ")[0]
        if date not in shown_dates:
            shown_dates.add(date)
            temp = item["main"]["temp"]
            cond = item["weather"][0]["description"]
            future_temps.append(temp)
            future_conditions.append(cond)
        if len(shown_dates) == max_days:
            break

    return {
        "past_temps": past_temps,
        "current_temp": current_temp,
        "future_temps": future_temps,
        "city_name": city,
        "humidity": humidity,
        "wind_speed": wind_speed,
        "icon": icon,
        "condition": condition,
        "future_conditions": future_conditions
    }

Cell 4: City Selection Dropdown

In [40]:
def select_city(city):
    global weather_data
    weather_data = get_weather(city)

interact(select_city, city=Dropdown(options=cities, description='Select City:'));

interactive(children=(Dropdown(description='Select City:', options=('Dhaka', 'Chittagong', 'Khulna', 'Sylhet',…

Cell 5: Prepare Current Weather Output

In [41]:
if weather_data:
    past_temps = weather_data["past_temps"]
    current_temp = weather_data["current_temp"]
    future_temps = weather_data["future_temps"]
    city_name = weather_data["city_name"]
    humidity = weather_data["humidity"]
    wind_speed = weather_data["wind_speed"]
    icon = weather_data["icon"]
    condition = weather_data["condition"]
    future_conditions = weather_data["future_conditions"]

    current_weather_str = f"""
🌤 CURRENT WEATHER - {city_name}
Temperature: {current_temp} °C | Humidity: {humidity}% | Wind: {wind_speed} m/s | Condition: {condition}
"""
    current_weather_icon = Image(ICON_URL.format(icon))

Cell 6: Prepare Past 3 Days Output

In [42]:
past_str = "\n⏪ PAST 3 DAYS (Estimated)\n"
for i, temp in enumerate(past_temps):
    past_str += f"Date: {(datetime.now() - timedelta(days=3-i)).strftime('%Y-%m-%d')} | Temp: {round(temp,1)} °C\n"

Cell 7: Prepare Next 7 Days Forecast

In [43]:
future_str = "\n⏩ NEXT 7 DAYS FORECAST\n"
for i in range(len(future_temps)):
    future_str += f"Date: {(datetime.now() + timedelta(days=i+1)).strftime('%Y-%m-%d')} | Temp: {round(future_temps[i],1)} °C | Condition: {future_conditions[i]}\n"

Cell 8: Save CSV

In [44]:
filename = f"{city_name}_weather_report.csv"
with open(filename, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Date", "Temperature (°C)", "Condition"])
    for i in range(3):
        writer.writerow([(datetime.now()-timedelta(days=3-i)).strftime("%Y-%m-%d"), round(past_temps[i],1), "Estimated"])
    writer.writerow([datetime.now().strftime("%Y-%m-%d"), current_temp, condition])
    for i in range(len(future_temps)):
        writer.writerow([(datetime.now() + timedelta(days=i+1)).strftime("%Y-%m-%d"), round(future_temps[i],1), future_conditions[i]])

csv_str = f"✅ CSV saved: {filename}"

Cell 9: Temperature Stats & Trend

In [45]:
temps = past_temps + [current_temp] + future_temps
avg_temp = sum(temps)/len(temps)
trend = "Stable"
if sum(future_temps)/len(future_temps) > sum(past_temps)/len(past_temps):
    trend = "Rising"
elif sum(future_temps)/len(future_temps) < sum(past_temps)/len(past_temps):
    trend = "Falling"

stats_str = f"📊 Temperature Stats: Avg: {avg_temp:.1f}°C | Min: {min(temps):.1f}°C | Max: {max(temps):.1f}°C | Trend: {trend}"

Cell 10: Prepare Alerts

In [46]:
alerts = []
if current_temp > 35:
    alerts.append("🔥 Heat Alert")
if current_temp < 5:
    alerts.append("❄ Cold Alert")
if humidity > 85:
    alerts.append("💧 High Humidity Alert")
if wind_speed > 15:
    alerts.append("💨 Strong Wind Alert")

alert_str = "\n⚠ WEATHER ALERTS:\n" + "\n".join(alerts) if alerts else "✅ No alerts today."

Cell 11: Prepare Forecast Table

In [47]:
table_html = "<table border='1' style='border-collapse:collapse; padding:5px;'>"
table_html += "<tr><th>Date</th><th>Temperature (°C)</th><th>Condition</th></tr>"
for i, temp in enumerate(past_temps):
    table_html += f"<tr><td>{(datetime.now()-timedelta(days=3-i)).strftime('%Y-%m-%d')}</td><td>{round(temp,1)}</td><td>Estimated</td></tr>"
table_html += f"<tr><td>{datetime.now().strftime('%Y-%m-%d')}</td><td>{current_temp}</td><td>{condition}</td></tr>"
for i in range(len(future_temps)):
    table_html += f"<tr><td>{(datetime.now()+timedelta(days=i+1)).strftime('%Y-%m-%d')}</td><td>{round(future_temps[i],1)}</td><td>{future_conditions[i]}</td></tr>"
table_html += "</table>"

Cell 12: Prepare Temperature Graph

In [48]:
labels = ["Past-3","Past-2","Past-1","Current"] + [f"Future-{i+1}" for i in range(len(future_temps))]
temps_plot = past_temps + [current_temp] + future_temps

fig_temp = go.Figure()
fig_temp.add_trace(go.Bar(x=labels, y=temps_plot, name="Temperature (°C)", marker_color="skyblue"))
fig_temp.add_trace(go.Scatter(x=labels, y=temps_plot, mode="lines+markers", name="Trend", line=dict(color="red")))
fig_temp.update_layout(title=f"Temperature Trend - {city_name}", yaxis_title="Temperature (°C)")

Cell 13: Prepare Humidity & Wind Graph

In [49]:
labels_humidity = labels
humidity_values = [round(humidity*0.95), round(humidity*0.97), round(humidity*0.99), humidity] + [round(humidity*1.02) for i in range(len(future_temps))]
wind_values = [round(wind_speed*0.95), round(wind_speed*0.97), round(wind_speed*0.99), wind_speed] + [round(wind_speed*1.05) for i in range(len(future_temps))]

fig_humidity = go.Figure()
fig_humidity.add_trace(go.Bar(x=labels_humidity, y=humidity_values, name="Humidity (%)", marker_color="lightgreen"))
fig_humidity.add_trace(go.Scatter(x=labels_humidity, y=wind_values, mode="lines+markers", name="Wind Speed (m/s)", line=dict(color="orange")))
fig_humidity.update_layout(title=f"Humidity & Wind Trend - {city_name}", yaxis_title="Values")

Cell 14: Prepare Combined Temp + Humidity Graph

In [50]:
fig_combined = go.Figure()
fig_combined.add_trace(go.Bar(x=labels, y=temps_plot, name="Temp (°C)", marker_color="skyblue"))
fig_combined.add_trace(go.Scatter(x=labels_humidity, y=humidity_values, mode="lines+markers", name="Humidity (%)", line=dict(color="green")))
fig_combined.update_layout(title=f"Temp & Humidity Combined - {city_name}", yaxis_title="Values")

Cell 15: Prepare Weather Summary Card

In [51]:
summary_html = f"""
<div style="border:2px solid #2fa8ff; padding:15px; border-radius:10px; width:400px; background:#f0f8ff;">
<h3>Weather Summary - {city_name}</h3>
<p>Current Temp: {current_temp} °C</p>
<p>Condition: {condition}</p>
<p>Humidity: {humidity}%</p>
<p>Wind Speed: {wind_speed} m/s</p>
<img src="{ICON_URL.format(icon)}" width="80"/>
</div>
"""

Cell 16: Prepare CSV Overview

In [52]:
df_overview = pd.read_csv(filename)

Cell 17: Prepare Weather Icons

In [53]:
icon_html = "<div style='display:flex; gap:15px;'>"
icon_html += f"<div><img src='{ICON_URL.format(icon)}'><br>Current</div>"
for i, cond in enumerate(future_conditions):
    icon_html += f"<div><img src='{ICON_URL.format(icon)}'><br>Day {i+1}</div>"
icon_html += "</div>"

Cell 18: Prepare Average Temperature Graph

In [54]:
avg_temp_past = sum(past_temps)/len(past_temps)
avg_temp_future = sum(future_temps)/len(future_temps)

fig_avg = go.Figure()
fig_avg.add_trace(go.Bar(x=["Past 3 Days Avg", "Future 7 Days Avg"], y=[avg_temp_past, avg_temp_future], marker_color=["lightblue","orange"]))
fig_avg.update_layout(title="Average Temperature Comparison", yaxis_title="Temp (°C)")

Cell 19: (Optimized Final Display Dashboard)

In [56]:
from IPython.display import display, HTML

# Wrap everything in a single dashboard HTML
dashboard_html = f"""
<div style="font-family:Arial, sans-serif;">

<!-- Current Weather + Icon -->
<div style="display:flex; align-items:center; gap:20px; margin-bottom:20px;">
    <div>
        <h2>🌤 CURRENT WEATHER - {city_name}</h2>
        <p>Temperature: {current_temp} °C</p>
        <p>Condition: {condition}</p>
        <p>Humidity: {humidity}% | Wind: {wind_speed} m/s</p>
    </div>
    <div><img src="{ICON_URL.format(icon)}" width="100"></div>
</div>

<!-- Past & Future Temperatures -->
<div style="display:flex; gap:50px; margin-bottom:20px;">
    <div>
        <h3>⏪ Past 3 Days (Estimated)</h3>
        <pre>{past_str}</pre>
    </div>
    <div>
        <h3>⏩ Next 7 Days Forecast</h3>
        <pre>{future_str}</pre>
    </div>
</div>

<!-- Stats & Alerts -->
<div style="margin-bottom:20px;">
    <h3>📊 Temperature Stats & Trend</h3>
    <p>{stats_str}</p>
    <h3>⚠ Weather Alerts</h3>
    <p>{alert_str}</p>
</div>

<!-- Forecast Table -->
<div style="margin-bottom:20px;">
    <h3>📅 Forecast Table</h3>
    {table_html}
</div>

<!-- Graphs -->
<div style="display:flex; gap:50px; flex-wrap:wrap; margin-bottom:20px;">
    <div style="flex:1; min-width:400px;">
        <h3>🌡 Temperature Trend</h3>
        {fig_temp.to_html(include_plotlyjs='cdn')}
    </div>
    <div style="flex:1; min-width:400px;">
        <h3>💧 Humidity & Wind Trend</h3>
        {fig_humidity.to_html(include_plotlyjs='cdn')}
    </div>
    <div style="flex:1; min-width:400px;">
        <h3>🌡 + 💧 Combined Temp & Humidity</h3>
        {fig_combined.to_html(include_plotlyjs='cdn')}
    </div>
    <div style="flex:1; min-width:400px;">
        <h3>📊 Avg Temp Comparison</h3>
        {fig_avg.to_html(include_plotlyjs='cdn')}
    </div>
</div>

<!-- Summary Card -->
<div style="margin-bottom:20px;">
    <h3>📝 Summary Card</h3>
    {summary_html}
</div>

<!-- Weather Icons -->
<div style="margin-bottom:20px;">
    <h3>🌤 Weather Icons</h3>
    {icon_html}
</div>

<!-- CSV Overview -->
<div style="margin-bottom:20px;">
    <h3>📁 CSV Overview</h3>
    {df_overview.to_html(index=False)}

</div>

</div>
"""

# Display the complete dashboard
display(HTML(dashboard_html))

Date,Temperature (°C),Condition
2025-12-13,8.4,Estimated
2025-12-14,9.6,Estimated
2025-12-15,10.8,Estimated
2025-12-16,11.99,overcast clouds
2025-12-17,12.1,light rain
2025-12-18,8.2,broken clouds
2025-12-19,9.8,light rain
2025-12-20,9.3,overcast clouds
2025-12-21,6.3,clear sky
2025-12-22,6.8,broken clouds
